# Pydeck Earth Engine Introduction

This is an introduction to using [Pydeck](https://pydeck.gl) and [Deck.gl](https://deck.gl) with [Google Earth Engine](https://earthengine.google.com/) in Jupyter Notebooks.

If you wish to run this locally, you'll need to install some dependencies. Installing into a new Conda environment is recommended. To create and enter the environment, run:
```
conda create -n pydeck-ee -c conda-forge python jupyter notebook pydeck earthengine-api requests -y
conda activate pydeck-ee

# Install the pydeck-earthengine-layers package from pip
pip install pydeck-earthengine-layers

jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck
jupyter nbextension enable --sys-prefix --py pydeck
```
then open Jupyter Notebook with `jupyter notebook`.

Now in a Python Jupyter Notebook, let's first import required packages:

In [1]:
from pydeck_earthengine_layers import EarthEngineLayer
import pydeck as pdk
import ee

## Authentication

### Authenticate with Earth Engine

Using Earth Engine requires authentication. If you don't have a Google account approved for use with Earth Engine, you'll need to request access. For more information and to sign up, go to https://signup.earthengine.google.com/.

If you haven't used Earth Engine in Python before, you'll need to run the following authentication command. If you've previously authenticated in Python or the command line, you can skip the next line.

Note that this creates a prompt which waits for user input. If you don't see a prompt, you may need to authenticate on the command line with `earthengine authenticate` and then return here, skipping the Python authentication.

In [2]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

### Initialize Earth Engine

The above authentication step creates credentials that are stored on your local computer. Those credentials need to be loaded so that Earth Engine and Pydeck will work.

## Power plants visualization



In [3]:
# Load the FeatureCollection
table = ee.FeatureCollection("WRI/GPPD/power_plants")

In [4]:
# Create color palette
fuel_color = ee.Dictionary({
  'Coal': '000000',
  'Oil': '593704',
  'Gas': 'BC80BD',
  'Hydro': '0565A6',
  'Nuclear': 'E31A1C',
  'Solar': 'FF7F00',
  'Waste': '6A3D9A',
  'Wind': '5CA2D1',
  'Geothermal': 'FDBF6F',
  'Biomass': '229A00'
})

In [5]:
# List of fuels to add to the map
fuels = ['Coal', 'Oil', 'Gas', 'Hydro', 'Nuclear', 'Solar', 'Waste',
    'Wind', 'Geothermal', 'Biomass']

In [6]:
def add_style(point):
    """Computes size from capacity and color from fuel type.
    
    Args:
        - point: (ee.Geometry.Point) A Point
        
    Returns:
        (ee.Geometry.Point): Input point with added style dictionary
    """
    size = ee.Number(point.get('capacitymw')).sqrt().divide(10).add(2)
    color = fuel_color.get(point.get('fuel1'))
    return point.set('styleProperty', ee.Dictionary({'pointSize': size, 'color': color}))

In [7]:
# Make a FeatureCollection out of the power plant data table
pp = ee.FeatureCollection(table).map(add_style)

In [8]:
# Create a layer for each fuel type
layers = []
for fuel in fuels:
    layer = EarthEngineLayer(
        pp.filter(ee.Filter.eq('fuel1', fuel)).style(styleProperty='styleProperty', neighborhood=50),
        id=fuel,
        opacity=0.65,
        library_url='https://unpkg.com/@unfolded.gl/earthengine-layers@1.0.0-beta.3/dist/pydeck_layer_module.min.js'
    )
    layers.append(layer)

Then just pass this layer to a `pydeck.Deck` instance, and call `.show()` to create a map:

In [9]:
view_state = pdk.ViewState(latitude=36, longitude=-53, zoom=3)
r = pdk.Deck(
    layers=layers, 
    initial_view_state=view_state
)
r.show()

DeckGLWidget(custom_libraries=[{'libraryName': 'EarthEngineLayerLibrary', 'resourceUri': 'https://unpkg.com/@u…